In [1]:
import pandas as pd
import numpy as np
import us

In [2]:
def state_to_fips(state_name_or_abbreviation):
    """
    Converts a US state name or abbreviation to its FIPS code.

    Args:
        state_name_or_abbreviation: The name or abbreviation of the state.

    Returns:
        The FIPS code of the state as a string, or None if not found.
    """
    state = us.states.lookup(state_name_or_abbreviation)
    if state:
        return state.fips
    else:
        return None

### State Popultion Cleaning

In [3]:
state_pops = pd.read_csv('../data/scm_covariates/original_data/state_pops.csv')
state_pops["STATEFIPS"] = state_pops["Geographic Area"].apply(state_to_fips)
state_pops.drop(columns = ["Geographic Area"], inplace = True)
state_pops.dropna(inplace = True)
state_pops.head()

,2020,2021,2022,2023,2024,STATEFIPS
0,"5,033,094","5,049,196","5,076,181","5,117,673","5,157,699",01
1,"733,017","734,420","734,442","736,510","740,133",02
2,"7,187,135","7,274,078","7,377,566","7,473,027","7,582,384",04
3,"3,014,546","3,026,870","3,047,704","3,069,463","3,088,354",05
4,"39,521,958","39,142,565","39,142,414","39,198,693","39,431,263",06


In [4]:
state_pops = state_pops.melt(id_vars=["STATEFIPS"], var_name="Year", value_name="Population")

# Convert Year to integer for better readability
state_pops["Year"] = state_pops["Year"].astype(int)
state_pops

,STATEFIPS,Year,Population
0,01,2020,"5,033,094"
1,02,2020,"733,017"
2,04,2020,"7,187,135"
3,05,2020,"3,014,546"
4,06,2020,"39,521,958"
...,...,...,...
245,51,2024,"8,811,195"
246,53,2024,"7,958,180"
247,54,2024,"1,769,979"
248,55,2024,"5,960,975"


In [5]:
state_pops["Population"] = state_pops['Population'].str.replace(',', '')
state_pops["Population"] = state_pops["Population"].astype(int)
state_pops.head()

,STATEFIPS,Year,Population
0,01,2020,5033094
1,02,2020,733017
2,04,2020,7187135
3,05,2020,3014546
4,06,2020,39521958


In [6]:
state_pops.to_csv("../data/scm_covariates/cleaned_data/state_pops_2020-2024.csv")

### Child Poverty Cleaning

In [7]:
child_poverty = pd.read_csv('../data/scm_covariates/original_data/child_poverty.csv')
child_poverty

,LocationType,Location,TimeFrame,DataFormat,Data
0,Nation,United States,2000,Number,12209000
1,State,Alabama,2000,Number,233000
2,State,Alaska,2000,Number,25000
3,State,Arizona,2000,Number,310000
4,State,Arkansas,2000,Number,168000
...,...,...,...,...,...
2433,State,Arkansas,2023,Percent,0.21
2434,State,Arizona,2023,Percent,0.15
2435,State,Alaska,2023,Percent,0.12
2436,State,Alabama,2023,Percent,0.21


In [8]:
child_poverty = child_poverty[(child_poverty['DataFormat'] == "Percent") & (child_poverty['LocationType'] == "State")]
child_poverty

,LocationType,Location,TimeFrame,DataFormat,Data
54,State,Wyoming,2000,Percent,0.15
55,State,Wisconsin,2000,Percent,0.12
56,State,West Virginia,2000,Percent,0.26
57,State,Washington,2000,Percent,0.16
58,State,Virginia,2000,Percent,0.13
...,...,...,...,...,...
2432,State,California,2023,Percent,0.15
2433,State,Arkansas,2023,Percent,0.21
2434,State,Arizona,2023,Percent,0.15
2435,State,Alaska,2023,Percent,0.12


In [9]:
child_poverty = child_poverty.drop(columns = ['LocationType', 'DataFormat'])
child_poverty.rename(columns={"Location": "STATEFIPS", "TimeFrame": "YEAR", "Data": "Child Poverty Rate"}, inplace=True)
child_poverty

,STATEFIPS,YEAR,Child Poverty Rate
54,Wyoming,2000,0.15
55,Wisconsin,2000,0.12
56,West Virginia,2000,0.26
57,Washington,2000,0.16
58,Virginia,2000,0.13
...,...,...,...
2432,California,2023,0.15
2433,Arkansas,2023,0.21
2434,Arizona,2023,0.15
2435,Alaska,2023,0.12


In [10]:
child_poverty["Child Poverty Rate"] = child_poverty["Child Poverty Rate"].astype(float)
child_poverty

,STATEFIPS,YEAR,Child Poverty Rate
54,Wyoming,2000,0.15
55,Wisconsin,2000,0.12
56,West Virginia,2000,0.26
57,Washington,2000,0.16
58,Virginia,2000,0.13
...,...,...,...
2432,California,2023,0.15
2433,Arkansas,2023,0.21
2434,Arizona,2023,0.15
2435,Alaska,2023,0.12


In [11]:
child_poverty["STATEFIPS"] = child_poverty["STATEFIPS"].apply(state_to_fips)
child_poverty

,STATEFIPS,YEAR,Child Poverty Rate
54,56,2000,0.15
55,55,2000,0.12
56,54,2000,0.26
57,53,2000,0.16
58,51,2000,0.13
...,...,...,...
2432,06,2023,0.15
2433,05,2023,0.21
2434,04,2023,0.15
2435,02,2023,0.12


In [12]:
child_poverty.to_csv("../data/scm_covariates/cleaned_data/child_poverty_rates.csv")

### 2020 Biden Margin Cleaning

In [13]:
biden_margins = pd.read_csv('../data/scm_covariates/original_data/2020_results.csv')
biden_margins

,state,called,final,dem_votes,rep_votes,other_votes,dem_percent,rep_percent,other_percent,dem_this_margin,...,stateid,EV,X,Y,State_num,Center_X,Center_Y,Unnamed: 19,2016 Margin,Total 2016 Votes
0,U.S. Total,D,Yes,"81,282,916","74,223,369","2,891,441",51.3%,46.9%,1.8%,4.5%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.020995,"136,639,848"
1,15 Key Battlegrounds,NaN,NaN,"31,908,248","33,002,287","950,354",48.4%,50.1%,1.4%,-1.7%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.036220,"56,209,173"
2,Non-Battlegrounds,NaN,NaN,"49,374,668","41,221,082","1,941,087",53.4%,44.5%,2.1%,8.8%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060978,"80,430,331"
3,15 Key Battlegrounds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arizona,D,Yes,"1,672,143","1,661,686","53,497",49.4%,49.1%,1.6%,0.3%,...,AZ,11.0,2.0,2.0,4.0,205.0,374.0,NaN,-0.035456,"2,573,165"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Vermont,D,Yes,"242,820","112,704","11,904",66.1%,30.7%,3.2%,35.4%,...,VT,3.0,10.0,6.0,50.0,873.0,134.0,NaN,0.264084,"315,067"
57,Virginia,D,Yes,"2,413,568","1,962,430","84,526",54.1%,44.0%,1.9%,10.1%,...,VA,13.0,8.0,3.0,51.0,809.0,299.0,NaN,0.053212,"3,982,752"
58,Washington,D,Yes,"2,369,612","1,584,651","133,368",58.0%,38.8%,3.3%,19.2%,...,WA,12.0,1.0,5.0,53.0,128.0,58.0,NaN,0.157060,"3,365,644"
59,West Virginia,R,Yes,"235,984","545,382","13,286",29.7%,68.6%,1.7%,-38.9%,...,WV,5.0,7.0,3.0,54.0,765.0,284.0,NaN,-0.420727,"713,051"


In [14]:
biden_margins['STATEFIPS'] = biden_margins['state'].apply(state_to_fips)
biden_margins = biden_margins[["STATEFIPS", 'dem_this_margin']]

In [15]:
biden_margins = biden_margins[~biden_margins["STATEFIPS"].isna()]
biden_margins.head()

,STATEFIPS,dem_this_margin
4,04,0.3%
5,12,-3.4%
6,13,0.2%
7,19,-8.2%
9,26,2.8%


In [16]:
biden_margins['biden_2020_margin'] = biden_margins['dem_this_margin'].str.strip("%").astype(float)
biden_margins = biden_margins[["STATEFIPS", 'biden_2020_margin']]
biden_margins["YEAR"] = 2020
biden_margins['biden_2020_margin'] /= 100
biden_margins.head()

,STATEFIPS,biden_2020_margin,YEAR
4,04,0.003,2020
5,12,-0.034,2020
6,13,0.002,2020
7,19,-0.082,2020
9,26,0.028,2020


In [17]:
biden_margins.to_csv("../data/scm_covariates/cleaned_data/Biden_2020_margins.csv")

### TANF Application Data

In [18]:
from os import listdir
from os.path import isfile, join

dir = '../data/scm_covariates/original_data/TANF_application_data/'

onlyfiles = [f for f in listdir(dir) if isfile(join(dir, f))]

In [19]:
tanf_df = pd.DataFrame()

for year in range(2005, 2022):

    # Load the Excel file
    for i in onlyfiles:
         if str(year) in i:
            file_path = i
            break
    # Read the first sheet
    xls = pd.ExcelFile(dir + file_path)
    df = xls.parse(xls.sheet_names[0])

    # Rename columns based on the third row (index 2)
    df.columns = df.iloc[2]

    # Drop the first three rows and reset index
    df = df[3:].reset_index(drop=True)

    df_columns = df.columns
    for col in df_columns:
        if "Average" in str(col):
            avg_col = col

    # Keep only the "State" and "Average FY 2018" columns
    df_filtered = df[["State", avg_col]].rename(columns={avg_col: "Monthly TANF Applications"})

    # Convert "Average FY 2018" to numeric values
    df_filtered["Monthly TANF Applications"] = pd.to_numeric(df_filtered["Monthly TANF Applications"], errors="coerce")
    df_filtered = df_filtered.dropna()

    df_filtered["State"] = df_filtered["State"].apply(state_to_fips)

    # Drop any NaN values
    df_filtered = df_filtered.dropna()

    df_filtered["YEAR"] = year

    tanf_df = pd.concat([tanf_df, df_filtered])

tanf_df.rename(columns = {"State": "STATEFIPS"}, inplace=True)
tanf_df

2,STATEFIPS,Monthly TANF Applications,YEAR
1,01,1887.000000,2005
2,02,966.000000,2005
3,04,8646.000000,2005
4,05,2822.000000,2005
5,06,32766.000000,2005
...,...,...,...
50,51,2824.416667,2021
51,53,5019.750000,2021
52,54,443.750000,2021
53,55,1250.333333,2021


In [20]:
tanf_df.to_csv("../data/scm_covariates/cleaned_data/tanf_application_numbers.csv")

### State Poverty Totals

In [21]:
import pandas as pd
import re

# Load the Excel file
file_path = "../data/scm_covariates/original_data/state_poverty_levels.xlsx"
xls = pd.ExcelFile(file_path)

# Load the sheet into a dataframe
df = pd.read_excel(xls, sheet_name="pov19")

# Identify header rows based on known pattern (years in the first column)
header_rows = df[df.iloc[:, 0].astype(str).str.match(r"\d{4}", na=False)].index

# Function to extract only the year from the string
def extract_year(year_str):
    match = re.search(r"\d{4}", str(year_str))
    return int(match.group()) if match else None

# Initialize an empty list to store the processed dataframes
all_data = []

# Extract tables based on detected header rows
for i, start_row in enumerate(header_rows):
    year = extract_year(df.iloc[start_row, 0])  # Extract the year
    if year is None:
        continue
    
    start_data_row = start_row + 2  # Data starts two rows after the year
    
    # Determine the end row for this table
    end_row = header_rows[i + 1] if i + 1 < len(header_rows) else len(df)
    
    # Extract table and assign column names
    table = df.iloc[start_data_row:end_row].copy()
    table.columns = df.iloc[start_data_row - 1]  # Use row above data as header
    table = table.iloc[1:].reset_index(drop=True)  # Remove old header row
    
    # Add year column
    table["Year"] = year
    
    all_data.append(table)

# Combine all tables into one
poverty = pd.concat(all_data, ignore_index=True)

# Display the first few rows of the cleaned dataframe
poverty


,State,Total population,Number in poverty,Margin of error (±) (29),Percent in poverty,Margin of error (±) (29),Year
0,Alaska,717,72,11,10.1,1.5,2023
1,Arizona,7210,861,160,11.9,2.2,2023
2,Arkansas,3011,416,56,13.8,1.9,2023
3,California,38550,4568,329,11.9,0.8,2023
4,Colorado,5876,498,98,8.5,1.7,2023
...,...,...,...,...,...,...,...
2297,West Virginia,1952,297,81,15.2,3.8,1980
2298,Wisconsin,4724,403,94,8.5,1.8,1980
2299,Wyoming,468,49,33,10.4,6.6,1980
2300,"Source: U.S. Census Bureau, Current Population...",NaN,NaN,NaN,NaN,NaN,1980


In [22]:
poverty["State"] = poverty["State"].apply(state_to_fips)
poverty.dropna(inplace=True)

In [23]:
poverty.rename(columns = {"Year": "YEAR", "State": "STATEFIPS"}, inplace= True)
poverty.drop(columns = ["Margin of error (±) (29)"], inplace=True)
for col in ["Total population", "Number in poverty"]:
    poverty[col] *= 1000

In [24]:
poverty

,STATEFIPS,Total population,Number in poverty,Percent in poverty,YEAR
0,02,717000,72000,10.1,2023
1,04,7210000,861000,11.9,2023
2,05,3011000,416000,13.8,2023
3,06,38550000,4568000,11.9,2023
4,08,5876000,498000,8.5,2023
...,...,...,...,...,...
2295,51,5204000,647000,12.4,1980
2296,53,4223000,538000,12.7,1980
2297,54,1952000,297000,15.2,1980
2298,55,4724000,403000,8.5,1980


In [25]:
poverty.to_csv("../data/scm_covariates/cleaned_data/poverty_stats.csv")

### Number applying for TANF over number in poverty

In [26]:
tanf_and_poverty_df = pd.merge(tanf_df, poverty, on = ["YEAR", "STATEFIPS"])
tanf_and_poverty_df

,STATEFIPS,Monthly TANF Applications,YEAR,Total population,Number in poverty,Percent in poverty
0,02,966.000000,2005,657000,66000,10
1,04,8646.000000,2005,6025000,917000,15.2
2,05,2822.000000,2005,2756000,382000,13.8
3,06,32766.000000,2005,35840000,4716000,13.2
4,08,641.000000,2005,4629000,530000,11.4
...,...,...,...,...,...,...
926,51,2824.416667,2021,8489000,831000,9.8
927,53,5019.750000,2021,7670000,587000,7.7
928,54,443.750000,2021,1754000,296000,16.9
929,55,1250.333333,2021,5910000,551000,9.3


In [27]:
tanf_and_poverty_df["TANF apps / Ind in poverty"] = tanf_and_poverty_df['Monthly TANF Applications'] / tanf_and_poverty_df["Number in poverty"]
tanf_and_poverty_df.drop_duplicates(["STATEFIPS", "YEAR"], inplace=True)
tanf_and_poverty_df

,STATEFIPS,Monthly TANF Applications,YEAR,Total population,Number in poverty,Percent in poverty,TANF apps / Ind in poverty
0,02,966.000000,2005,657000,66000,10,0.014636
1,04,8646.000000,2005,6025000,917000,15.2,0.009429
2,05,2822.000000,2005,2756000,382000,13.8,0.007387
3,06,32766.000000,2005,35840000,4716000,13.2,0.006948
4,08,641.000000,2005,4629000,530000,11.4,0.001209
...,...,...,...,...,...,...,...
926,51,2824.416667,2021,8489000,831000,9.8,0.003399
927,53,5019.750000,2021,7670000,587000,7.7,0.008552
928,54,443.750000,2021,1754000,296000,16.9,0.001499
929,55,1250.333333,2021,5910000,551000,9.3,0.002269


In [28]:
tanf_and_poverty_df.to_csv("../data/scm_covariates/cleaned_data/tanf_and_poverty_df.csv")

### Concatenate Covariates into a Single DataFrame

In [29]:
covariates = pd.merge(tanf_and_poverty_df, child_poverty, on = ["YEAR", "STATEFIPS"], how = "outer")
covariates[["STATEFIPS", "YEAR", "Child Poverty Rate", "TANF apps / Ind in poverty"]]
covariates = pd.merge(covariates, biden_margins, on = ["STATEFIPS", "YEAR"], how = 'outer')
covariates = covariates[['STATEFIPS', "YEAR", "TANF apps / Ind in poverty", "Child Poverty Rate", "biden_2020_margin"]]
covariates

,STATEFIPS,YEAR,TANF apps / Ind in poverty,Child Poverty Rate,biden_2020_margin
0,01,2000,NaN,0.21,NaN
1,01,2001,NaN,0.23,NaN
2,01,2002,NaN,0.24,NaN
3,01,2003,NaN,0.24,NaN
4,01,2004,NaN,0.23,NaN
...,...,...,...,...,...
1195,56,2019,0.004058,0.12,NaN
1196,56,2020,0.003283,NaN,-0.434
1197,56,2021,0.003346,0.13,NaN
1198,56,2022,NaN,0.14,NaN


In [30]:
covariates.to_csv('../data/scm_covariates/covariates.csv')